# Analyze A/B Test Results 

This project will assure you have mastered the subjects covered in the statistics lessons. We have organized the current notebook into the following sections: 

- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
## Introduction

A/B tests are very commonly performed by data analysts and data scientists. For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should:
- Implement the new webpage, 
- Keep the old webpage, or 
- Perhaps run the experiment longer to make their decision.

<a id='probability'></a>
## Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

### 1.1
Below is the description of the data, there are a total of 5 columns:

<center>

|Data columns|Purpose|Valid values|
| ------------- |:-------------| -----:|
|user_id|Unique ID|Int64 values|
|timestamp|Time stamp when the user visited the webpage|-|
|group|In the current A/B experiment, the users are categorized into two broad groups. <br>The `control` group users are expected to be served with `old_page`; and `treatment` group users are matched with the `new_page`. <br>However, **some inaccurate rows** are present in the initial data, such as a `control` group user is matched with a `new_page`. |`['control', 'treatment']`|
|landing_page|It denotes whether the user visited the old or new webpage.|`['old_page', 'new_page']`|
|converted|It denotes whether the user decided to pay for the company's product. Here, `1` means yes, the user bought the product.|`[0, 1]`|
</center>

<br>
<br>

**a.** Read in the dataset from the `ab_data.csv` file and take a look at the top few rows here:

In [2]:
df=pd.read_csv('ab_data.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ab_data.csv'

**b.** Find the number of rows in the dataset.

In [ ]:
df.shape[0]

**c.** The number of unique users in the dataset.

In [ ]:
df['user_id'].nunique()

**d.** The proportion of users converted.

In [ ]:
us_converted=(df.query('converted == 1')['user_id'].nunique())
us_converted/df.shape[0]

* The proportion of users converted ≈ 0.11944

**e.** The number of times when the "group" is `treatment` but "landing_page" is not a `new_page`.

In [ ]:
(df.query('(group == "treatment" and landing_page == "old_page") or (group == "control" and landing_page == "new_page")')['user_id'].count())

**f.** Do any of the rows have missing values?

In [ ]:
df.isnull().values.any().sum()

### 1.2  
In a particular row, the **group** and **landing_page** columns should have either of the following acceptable values:

|user_id| timestamp|group|landing_page|converted|
|---|---|---|---|---|
|XXXX|XXXX|`control`| `old_page`|X |
|XXXX|XXXX|`treatment`|`new_page`|X |


It means, the `control` group users should match with `old_page`; and `treatment` group users should matched with the `new_page`. 

However, for the rows where `treatment` does not match with `new_page` or `control` does not match with `old_page`, we cannot be sure if such rows truly received the new or old wepage.  



**a.** create a new dataset that meets the specifications. Store new dataframe in **df2**.

In [ ]:
# Remove the inaccurate rows, and store the result in a new dataframe df2
df2 = df.drop(df.query('(group == "treatment" and landing_page == "old_page") or (group == "control" and landing_page == "new_page")').index)

In [ ]:
# Double Check all of the incorrect rows were removed from df2 - 
# Output of the statement below should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

### 1.3  


**a.** How many unique **user_id**s are in **df2**?

In [ ]:
df2['user_id'].nunique()

**b.** There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2['user_id'].duplicated(keep=False)]['user_id']

**c.** Display the rows for the duplicate **user_id**? 

In [ ]:
df2[df2['user_id'] == 773192]

**d.** Remove **one** of the rows with a duplicate **user_id**, from the **df2** dataframe.

In [ ]:
# Remove one of the rows with a duplicate user_id..
# Hint: The dataframe.drop_duplicates() may not work in this case because the rows with duplicate user_id are not entirely identical. 
df2 = df2.drop(df2[(df2.user_id == 773192) & (df2['timestamp'] == '2017-01-14 02:55:59.590927')].index)
# Check again if the row with a duplicate user_id is deleted or not
df2[df2['user_id'] == 773192]

### 1.4  

**a.** What is the probability of an individual converting regardless of the page they receive?<br><br>




In [ ]:
us_converted_df2=(df2.query('converted == 1')['user_id'].nunique())
(us_converted_df2/df2.shape[0])*100

* The probability of an individual converting regardless of the page they receive ≈ 11.96%

**b.** Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
converted_control_df2 = (df2.query('group == "control" and converted == 1')['user_id'].nunique())
conv_rate_cont = (converted_control_df2/df2.query('group == "control"')['user_id'].nunique())
conv_rate_cont

* the probability they converted ≈ 0.1204

**c.** Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
converted_treatment_df2 = (df2.query('group == "treatment" and converted == 1')['user_id'].nunique())
conv_rate_treat = (converted_treatment_df2/(df2.query('group == "treatment"')['user_id'].nunique()))
conv_rate_treat

* the probability they converted ≈ 0.1188

> The probabilities we've computed in the points (b). and (c). above can also be treated as conversion rate. 
Calculate the actual difference  (`obs_diff`) between the conversion rates for the two groups. We will need that later.  

In [ ]:
# Calculate the actual difference (obs_diff) between the conversion rates for the two groups.
obs_diff = conv_rate_treat - conv_rate_cont
obs_diff

**d.** What is the probability that an individual received the new page?

In [ ]:
new_page_df2 = (df2.query('landing_page == "new_page"')['user_id'].nunique())
new_page_df2/df2.shape[0]

* ≈ 0.5001 received the new page





**e.** Considering results from parts (a) through (d) above,

>The probablity that users converted in both groups & probability of converting regardless of the "page" are pretty close to each other. 
so, there is no evidence that new_page leads to more conversions.

<a id='ab_test'></a>
## Part II - A/B Test

Since a timestamp is associated with each event, we could run a hypothesis test continuously as long as we observe the events. 

However, then the hard questions would be: 
- Do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  
- How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


### 2.1
For now, consider we need to make the decision just based on all the data provided.  

> Recall that we just calculated that the "converted" probability (or rate) for the old page is *slightly* higher than that of the new page (1.4.c). 

If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should be your null and alternative hypotheses (**$H_0$** and **$H_1$**)?  

You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the "converted" probability (or rate) for the old and new pages respectively.

>**$H_0$: $p_{new}$ - $p_{old}$ <= 0 <br>
 $H_1$: $p_{new}$ - $p_{old}$ > 0.**

### 2.2 - Null Hypothesis $H_0$ Testing
Under the null hypothesis $H_0$, assume that $p_{new}$ and $p_{old}$ are equal. Furthermore, assume that $p_{new}$ and $p_{old}$ both are equal to the **converted** success rate in the `df2` data regardless of the page. So, our assumption is: <br><br>
<center>
$p_{new}$ = $p_{old}$ = $p_{population}$
</center>

In this section, we will: 

- Simulate (bootstrap) sample data set for both groups, and compute the  "converted" probability $p$ for those samples. 


- Use a sample size for each group equal to the ones in the `df2` data.


- Compute the difference in the "converted" probability for the two samples above. 


- Perform the sampling distribution for the "difference in the converted probability" between the two simulated-samples over 10,000 iterations; and calculate an estimate. 




**a.** What is the **conversion rate** for $p_{new}$ under the null hypothesis? 

In [ ]:
p_new = (df2.query('converted == 1')['user_id'].nunique())/(df2['user_id'].nunique())
p_new

* conversion rate for  𝑝𝑛𝑒𝑤  under the null hypothesis ≈ 0.1196

**b.** What is the **conversion rate** for $p_{old}$ under the null hypothesis? 

In [ ]:
p_old = (df2.query('converted == 1')['user_id'].nunique())/(df2['user_id'].nunique())
p_old

* conversion rate for  𝑝𝑜𝑙𝑑  under the null hypothesis ≈ 0.1196

**c.** What is $n_{new}$, the number of individuals in the treatment group? <br><br>
 >The treatment group users are shown the new page.

In [ ]:
n_new = df2.query('landing_page == "new_page"')['user_id'].nunique()
n_new

**d.** What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old = df2.query('landing_page == "old_page"')['user_id'].nunique() 
n_old

**e. Simulate Sample for the `treatment` Group**<br> 
Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null hypothesis.  <br><br>
*Hint*: Use `numpy.random.choice()` method to randomly generate $n_{new}$ number of values. <br>
Store these $n_{new}$ 1's and 0's in the `new_page_converted` numpy array.


In [ ]:
# Simulate a Sample for the treatment Group
new_page_converted = np.random.choice([0,1],n_new, p=(p_new,1-p_new))
new_page_converted

**f. Simulate Sample for the `control` Group** <br>
Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null hypothesis. <br> Store these $n_{old}$ 1's and 0's in the `old_page_converted` numpy array.

In [ ]:
# Simulate a Sample for the control Group
old_page_converted = np.random.choice([0,1],n_old, p=(p_old,1-p_old))
old_page_converted

**g.** Find the difference in the "converted" probability $(p{'}_{new}$ - $p{'}_{old})$ for your simulated samples from the parts (e) and (f) above. 

In [ ]:
(new_page_converted.mean()) - (old_page_converted.mean())


**h. Sampling distribution** <br>
Re-create `new_page_converted` and `old_page_converted` and find the $(p{'}_{new}$ - $p{'}_{old})$ value 10,000 times using the same simulation process you used in parts (a) through (g) above. 

<br>
Store all  $(p{'}_{new}$ - $p{'}_{old})$  values in a NumPy array called `p_diffs`.

In [ ]:
# Sampling distribution 

p_diffs = []
new_converted_simulation = np.random.binomial(n_new, p_new, 10000)/n_new 
old_converted_simulation = np.random.binomial(n_old, p_old, 10000)/n_old 
p_diffs = new_converted_simulation - old_converted_simulation

**i. Histogram**<br> 
Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  <br><br>

Also, use `plt.axvline()` method to mark the actual difference observed  in the `df2` data (recall `obs_diff`), in the chart.  



In [ ]:
p_diffs = np.array(p_diffs)
plt.hist(p_diffs);

**j.** What proportion of the **p_diffs** are greater than the actual difference observed in the `df2` data?

In [ ]:
#creat null distribution
null_vals = np.random.normal(0, p_diffs.std(), p_diffs.size)
#plot null distribution
plt.hist(null_vals)
#draw a V line for the observed stat
plt.axvline(x=obs_diff,color ='orange');

In [ ]:
#proportion of the p_diffs are > than the actual difference observed
(null_vals > obs_diff).mean()

**k.** Please explain in words what you have just computed in part **j** above.  
 - What is this value called in scientific studies?  
 - What does this value signify in terms of whether or not there is a difference between the new and old pages? 

> p_old > **$α$** , we fail to reject the null. So, data shows : that the old page has a higher rate of convergence than the new page, {with a type I error rate of 0.05}.



**l. Using Built-in Methods for Hypothesis Testing**<br>
We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. 

Fill in the statements below to calculate the:
- `convert_old`: number of conversions with the old_page
- `convert_new`: number of conversions with the new_page
- `n_old`: number of individuals who were shown the old_page
- `n_new`: number of individuals who were shown the new_page


In [ ]:
import statsmodels.api as sm


# number of conversions with the old_page
convert_old = df2.query('landing_page == "old_page" and converted == 1')['user_id'].nunique()
# number of conversions with the new_page
convert_new = df2.query('landing_page == "new_page" and converted == 1')['user_id'].nunique()
# number of individuals who were shown the old_page
#n_old
# number of individuals who received new_page
#n_new
convert_old,convert_new,n_old,n_new

**m.** Now use `sm.stats.proportions_ztest()` to compute your test statistic and p-value.  [Here](https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html) is a helpful link on using the built in.

The syntax is: 
```bash
proportions_ztest(count_array, nobs_array, alternative='larger')
```
where, 
- `count_array` = represents the number of "converted" for each group
- `nobs_array` = represents the total number of observations (rows) in each group
- `alternative` = choose one of the values from `[‘two-sided’, ‘smaller’, ‘larger’]` depending upon two-tailed, left-tailed, or right-tailed respectively. 
>**Hint**: <br>
It's a two-tailed if you defined $H_1$ as $(p_{new} = p_{old})$. <br>
It's a left-tailed if you defined $H_1$ as $(p_{new} < p_{old})$. <br>
It's a right-tailed if you defined $H_1$ as $(p_{new} > p_{old})$. 

The built-in function above will return the z_score, p_value. 

---
### About the two-sample z-test
Recall that you have plotted a distribution `p_diffs` representing the
difference in the "converted" probability  $(p{'}_{new}-p{'}_{old})$  for your two simulated samples 10,000 times. 

Another way for comparing the mean of two independent and normal distribution is a **two-sample z-test**. You can perform the Z-test to calculate the Z_score, as shown in the equation below:

$$
Z_{score} = \frac{ (p{'}_{new}-p{'}_{old}) - (p_{new}  -  p_{old})}{ \sqrt{ \frac{\sigma^{2}_{new} }{n_{new}} + \frac{\sigma^{2}_{old} }{n_{old}}  } }
$$

where,
- $p{'}$ is the "converted" success rate in the sample
- $p_{new}$ and $p_{old}$ are the "converted" success rate for the two groups in the population. 
- $\sigma_{new}$ and $\sigma_{new}$ are the standard deviation for the two groups in the population. 
- $n_{new}$ and $n_{old}$ represent the size of the two groups or samples (it's same in our case)


>Z-test is performed when the sample size is large, and the population variance is known. The z-score represents the distance between the two "converted" success rates in terms of the standard error. 

Next step is to make a decision to reject or fail to reject the null hypothesis based on comparing these two values: 
- $Z_{score}$
- $Z_{\alpha}$ or $Z_{0.05}$, also known as critical value at 95% confidence interval.  $Z_{0.05}$ is 1.645 for one-tailed tests,  and 1.960 for two-tailed test. You can determine the $Z_{\alpha}$ from the z-table manually. 

Decide if your hypothesis is either a two-tailed, left-tailed, or right-tailed test. Accordingly, reject OR fail to reject the  null based on the comparison between $Z_{score}$ and $Z_{\alpha}$. We determine whether or not the $Z_{score}$ lies in the "rejection region" in the distribution. In other words, a "rejection region" is an interval where the null hypothesis is rejected iff the $Z_{score}$ lies in that region.

>Hint:<br>
For a right-tailed test, reject null if $Z_{score}$ > $Z_{\alpha}$. <br>
For a left-tailed test, reject null if $Z_{score}$ < $Z_{\alpha}$. 




Reference: 
- Example 9.1.2 on this [page](https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/09%3A_Two-Sample_Problems/9.01%3A_Comparison_of_Two_Population_Means-_Large_Independent_Samples), courtesy www.stats.libretexts.org

---


In [ ]:
import statsmodels.api as sm

z_score, p_value = sm.stats.proportions_ztest(np.array([convert_new,convert_old]),np.array([n_new,n_old]), alternative = 'larger')
print(z_score, p_value)

**n.** What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?<br><br>


> the z-score ≈ 1.311 < the critical value of 1.960,
So we fail to reject the null hypothesis that $p_{new}$ - $p_{old}$ $<= 0$ . Therefore, $p_{new}$ - $p_{old}$ $= 0$. <br> results agree with parts J. and K. 

<a id='regression'></a>
### Part III - A regression approach

### 3.1 
In this final part, we will see that the result we achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

**a.** Since each row in the `df2` data is either a conversion or no conversion, what type of regression should you be performing in this case?

>**response variable is categorical variable ,So we use Logistic Regression.**

**b.** The goal is to use **statsmodels** library to fit the regression model you specified in part **a.** above to see if there is a significant difference in conversion based on the page-type a customer receives. However, you first need to create the following two columns in the `df2` dataframe:
 1. `intercept` - It should be `1` in the entire column. 
 2. `ab_page` - It's a dummy variable column, having a value `1` when an individual receives the **treatment**, otherwise `0`.  

In [ ]:
df2['intercept'] = 1

df2['ab_page'] = pd.get_dummies(df['group']) ['treatment']
df2.head()


**c.** Use **statsmodels** to instantiate your regression model on the two columns you created in part (b). above, then fit the model to predict whether or not an individual converts. 


In [ ]:
logm = sm.Logit(df2['converted'], df2[['intercept','ab_page']])
result = logm.fit()

**d.** Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
result.summary2()

**e.** What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  

**Hints**: 
- What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**? 
- You may comment on if these hypothesis (Part II vs. Part III) are one-sided or two-sided. 
- You may also compare the current p-value with the Type I error rate (0.05).


>**The p-value associated with ab_page is 0.1899.** <br>  $H_0$ in c~e questions say that there is no difference between the T/C groups. $H_1$ say that there is a difference between them.
in Part II we assume old_page is better unless we prove that new_page is better at *$α$* of 5%. <br> So we have a different explanatory variable for each outcome.

**f.** Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

>**Looking back at the data we have 'timestamp' column. <br> Convergence rate may be affected by different times of the day or the  day it self when users view the website page. For timestamp variable, <br> we can also consider time as different day types {weekday, weekend} (categorical variable).**<br><br> 
* Disadavantage for adding additional terms into our regression model is that it will make the interpretation of the model more complex.
*  if new terms dependent with the our explanatory terms, Then we need to add a higher order term to help predict the result efficiently.

**g. Adding countries**<br> 
Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. 

1. You will need to read in the **countries.csv** dataset and merge together your `df2` datasets on the appropriate rows. You call the resulting dataframe `df_merged`. [Here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) are the docs for joining tables. 

2. Does it appear that country had an impact on conversion?  To answer this question, consider the three unique values, `['UK', 'US', 'CA']`, in the `country` column. Create dummy variables for these country columns. 
>**Hint:** Use `pandas.get_dummies()` to create dummy variables. **You will utilize two columns for the three dummy variables.** 

 Provide the statistical output as well as a written response to answer this question.

In [ ]:
# Read the countries.csv
countries = pd.read_csv('countries.csv')
countries.head()

In [ ]:
# Join with the df2 dataframe
# on : to keep the appropriate rows
# how : to add contry column at the end/right of the df
df_merged = df2.merge(countries, on ='user_id', how='left')
df_merged.head()

In [ ]:
# Create the necessary dummy variables
# we named new columns after seeing the result of dummies
df_merged[['CA','UK','US']] = pd.get_dummies(df_merged['country'])

In [ ]:
df_merged.head()

In [ ]:
# Fit your model, and summarize the results
df_merged['intercept'] = 1

#Create logistic regression model for conveted and country,using CA and old page as baseline (ab_page) holds new_page values
logmmerg = sm.Logit(df_merged['converted'], df_merged[['intercept','UK','US','ab_page']])
result = logmmerg.fit()
result.summary2()

In [ ]:
np.exp(0.0506),np.exp(0.0408),1/np.exp(-0.0149)

* RESULTS:

<br>
convergence ≈ 5.2% more likely to happen, for every unit increase with UK , holding all other varible constant.<br>
convergence ≈ 4.2% more to happen, for every unit increase in US , holding all other varible constant.<br>
convergence will be ≈ 1.5% more likely to happen, for every unit decrease in ab_page , holding all other varible constant.

**h. Fit your model and obtain the results**<br> 
Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if are there significant effects on conversion.  **Create the necessary additional columns, and fit the new model.** 





In [ ]:
#Create a new intereacton variable between new page and country US and UK
df_merged['ab_page*UK'] = df_merged['UK']*df_merged['ab_page']
df_merged['ab_page*US'] = df_merged['US']*df_merged['ab_page']

In [ ]:
#Create logistic regression for the new intereaction variable
logmmergint = sm.Logit(df_merged['converted'], df_merged[['intercept','UK','US','ab_page','ab_page*UK','ab_page*US']])
result = logmmergint.fit()
result.summary2()

In [ ]:
np.exp(result.params)

* RESULTS:

<br>
Convergence ≈ 1.19 % more likely to happen for the users in UK to those in CA, holding all other varibles constant.

Convergence ≈ 1.76 % more likely to happen for the users in US to those in CA, holding all other varibles constant.

Convergence will be ≈ 7.0% more likely to happen, for every unit decrease on ab_page, holding all other varibles constant.

Convergence ≈ 1.08 times more likely to happen for UK and new page users to those in CA and new page users, holding all other varibles constant.

Convergence ≈ 1.04 times more likely to happen for US and new page users to those in CA and new page users, holding all other varibles constant.
<br>
> 1.  We can see that the coefficient of ab_page is different from the coefficient of intereaction variable "ab_page*UK" and "ab_page*US" as well.
2. All  p-values are more than 0.05 except the intercept which is statistically significant.


>**the country where the user lives in is not significant on the convergence rate considering the page.**